In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

*Nutzungsschätzung: unter einer Minute auf einem Eagle r3-Prozessor (HINWEIS: Dies ist nur eine Schätzung. Ihre Laufzeit kann variieren.)*

## Hintergrund

Amplitudenverstärkung ist ein universeller Quantenalgorithmus oder eine Unterroutine, die verwendet werden kann, um eine quadratische Beschleunigung gegenüber einer Handvoll klassischer Algorithmen zu erzielen. [Grovers Algorithmus](https://arxiv.org/abs/quant-ph/9605043) war der erste, der diese Beschleunigung bei unstrukturierten Suchproblemen demonstrierte. Die Formulierung eines Groverschen Suchproblems erfordert eine Orakelfunktion, die einen oder mehrere Basiszustände als die Zustände markiert, die wir finden möchten, und einen Verstärkungsschaltkreis, der die Amplitude der markierten Zustände erhöht und folglich die verbleibenden Zustände unterdrückt.

Hier demonstrieren wir, wie man Grover-Orakel konstruiert und den [`grover_operator()`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.grover_operator) aus der Qiskit-Schaltkreisbibliothek verwendet, um einfach eine Groversche Suchinstanz einzurichten. Das Runtime `Sampler`-Primitiv ermöglicht die nahtlose Ausführung von Grover-Schaltkreisen.

## Anforderungen

Stellen Sie vor Beginn dieses Tutorials sicher, dass Sie Folgendes installiert haben:

* Qiskit SDK v1.4 oder neuer, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.36 oder neuer

## Setup

In [1]:
# Built-in modules
import math

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import grover_operator, MCMTGate, ZGate
from qiskit.visualization import plot_distribution
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler


def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [
            ind
            for ind in range(num_qubits)
            if rev_target.startswith("0", ind)
        ]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        if zero_inds:
            qc.x(zero_inds)
        qc.compose(MCMTGate(ZGate(), num_qubits - 1, 1), inplace=True)
        if zero_inds:
            qc.x(zero_inds)
    return qc

## Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden
Grovers Algorithmus benötigt ein [Orakel](/learning/courses/fundamentals-of-quantum-algorithms/grover-algorithm/introduction), das einen oder mehrere markierte Basiszustände spezifiziert, wobei "markiert" einen Zustand mit einer Phase von -1 bedeutet. Ein Controlled-Z-Gate oder seine mehrfach kontrollierte Verallgemeinerung über $N$ Qubits markiert den $2^{N}-1$-Zustand (`'1'`*$N$ Bit-String). Das Markieren von Basiszuständen mit einem oder mehreren `'0'` in der binären Darstellung erfordert das Anwenden von X-Gates auf die entsprechenden Qubits vor und nach dem Controlled-Z-Gate; dies entspricht einer offenen Kontrolle auf diesem Qubit. Im folgenden Code definieren wir ein Orakel, das genau das tut und einen oder mehrere Eingabe-Basiszustände markiert, die durch ihre Bit-String-Darstellung definiert sind. Das `MCMT`-Gate wird verwendet, um das mehrfach kontrollierte Z-Gate zu implementieren.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_brisbane'

### Specific Grover's instance

Now that we have the oracle function, we can define a specific instance of Grover search.  In this example we will mark two computational states out of the eight available in a three-qubit computational space:

In [3]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/c150298f-0.avif" alt="Output of the previous code cell" />

In [4]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/7baca7e2-99fc-4089-b5d8-30da56816a6a-0.avif" alt="Output of the previous code cell" />

In [5]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/d3a26fc9-9090-4527-a749-a412661260b6-0.avif" alt="Output of the previous code cell" />

### Grover operator

The built-in Qiskit `grover_operator()` takes an oracle circuit and returns a circuit that is composed of the oracle circuit itself and a circuit that amplifies the states marked by the oracle.  Here, we use the `decompose()` method the circuit to see the gates within the operator:

In [6]:
grover_op = grover_operator(oracle)
grover_op.decompose().draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/283d5265-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/d3a26fc9-9090-4527-a749-a412661260b6-0.avif)

### Grover-Operator

Der eingebaute Qiskit `grover_operator()` nimmt einen Orakel-Schaltkreis entgegen und gibt einen Schaltkreis zurück, der aus dem Orakel-Schaltkreis selbst und einem Schaltkreis besteht, der die vom Orakel markierten Zustände verstärkt. Hier verwenden wir die `decompose()`-Methode des Schaltkreises, um die Gates innerhalb des Operators zu sehen:

In [7]:
optimal_num_iterations = math.floor(
    math.pi
    / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/283d5265-0.avif)

Wiederholte Anwendungen dieses `grover_op`-Schaltkreises verstärken die markierten Zustände und machen sie zu den wahrscheinlichsten Bit-Strings in der Ausgabeverteilung des Schaltkreises. Es gibt eine optimale Anzahl solcher Anwendungen, die durch das Verhältnis markierter Zustände zur Gesamtzahl möglicher Berechnungszustände bestimmt wird:

In [8]:
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()
qc.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/4933ae44-0.avif" alt="Output of the previous code cell" />

### Vollständiger Grover-Schaltkreis
Ein vollständiges Grover-Experiment beginnt mit einem Hadamard-Gate auf jedem Qubit; dies erzeugt eine gleichmäßige Überlagerung aller Berechnungsbasisstände, gefolgt vom Grover-Operator (`grover_op`), der die optimale Anzahl von Malen wiederholt wird. Hier verwenden wir die `QuantumCircuit.power(INT)`-Methode, um den Grover-Operator wiederholt anzuwenden.

In [9]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

circuit_isa = pm.run(qc)
circuit_isa.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/c9a3020e-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/4933ae44-0.avif)

## Schritt 2: Problem für die Ausführung auf Quantenhardware optimieren

In [10]:
# To run on local simulator:
#   1. Use the StatevectorSampler from qiskit.primitives instead
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_isa]).result()
dist = result[0].data.meas.get_counts()

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/c9a3020e-0.avif)

## Schritt 3: Ausführen mit Qiskit-Primitiven
Amplitudenverstärkung ist ein Sampling-Problem, das für die Ausführung mit dem [`Sampler`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/sampler-v2)-Runtime-Primitiv geeignet ist.

Beachten Sie, dass die `run()`-Methode des [Qiskit Runtime `SamplerV2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/sampler-v2) ein Iterable von `primitive unified blocks (PUBs)` akzeptiert. Für den Sampler ist jedes PUB ein Iterable im Format `(circuit, parameter_values)`. Mindestens akzeptiert er jedoch eine Liste von Quantenschaltkreis(en).

In [11]:
plot_distribution(dist)

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/a5ef9913-0.avif" alt="Output of the previous code cell" />

## Schritt 4: Nachbearbeitung und Rückgabe des Ergebnisses im gewünschten klassischen Format